In [2]:
from sklearn.externals import joblib
import os
import json
import numpy as np
import math
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from collections import OrderedDict
from collections import defaultdict
from collections import Counter
from gensim.models import Word2Vec

In [3]:
predictiondict_1 = joblib.load(open("predictions_tablets_IT.p", "rb"))

In [4]:
predictiondict_2 = joblib.load(open("predictions_automobiles_IT.p", "rb"))

In [5]:
tablets_automobiles = ["tablets_IT_extra_data","automobiles_IT_extra_data"]

In [6]:
raw_data_list = []

for dirfile in tablets_automobiles:
        print("collecting comments from: ",dirfile)    
        for filename in os.listdir(dirfile):
            file = dirfile+"/"+filename
            if file[-4:] == "json":
                with open(file) as json_data:
                    d = json.load(json_data)
                    for comment in d["comments"]:
                        lower = comment["text"].lower().strip()
                        raw_data_list.append([comment["id"],d["video_id"],lower])

collecting comments from:  /media/moniek/Elements/Thesis/Make_normal_Embeddings/tablets_IT_extra_data
collecting comments from:  /media/moniek/Elements/Thesis/Make_normal_Embeddings/automobiles_IT_extra_data


In [7]:
print(len(predictiondict_2))

99328


In [8]:
predictiondict = {}

In [9]:
predictiondict.update(predictiondict_2)

In [10]:
print(len(predictiondict))

99328


In [11]:
predictiondict.update(predictiondict_1)

In [12]:
print(len(predictiondict))

212632


In [13]:
# create commentlists of positive, negative and neutral comments

labellist_pos_test = []
commentlist_pos_test =[]
labellist_neg_test = []
commentlist_neg_test =[]
labellist_neu_test = []
commentlist_neu_test =[]
for comment in raw_data_list:
    commentid, videoid_raw, comment_text = comment
    
    if commentid in predictiondict:
        
        label,videoid,comment_output = predictiondict[commentid]
        
        if videoid == videoid_raw:
            
            raw_comment = comment_text
            comment = raw_comment.lower().strip()
            words = comment.split()
            
            if label == "positive":
                commentlist_pos_test.append(words)
                labellist_pos_test.append(label)
            elif label == "negative":
                commentlist_neg_test.append(words)
                labellist_neg_test.append(label)
            elif label == "neutral":
                commentlist_neu_test.append(words)
                labellist_neu_test.append(label)
        else:
            print(commentid, videoid_raw, comment_text )
    else:
        print(commentid)

In [14]:
print(len(commentlist_pos_test))
print(len(commentlist_neg_test))
print(len(commentlist_neu_test))

print(len(commentlist_pos_test)+len(commentlist_neg_test)+len(commentlist_neu_test))

24516
9516
183570
217602


In [15]:
colnames =["lemma","POS","WordnetID","PosScore","NegScore","polarity","intensity"]
lexiconfile = pd.read_csv('sentix', sep="\t", header=None, names=colnames)

In [16]:
lexiconfile.head(2)

,lemma,POS,WordnetID,PosScore,NegScore,polarity,intensity
0,abile,a,1740,0.125,0.0,1.0,0.125
1,intelligente,a,1740,0.125,0.0,1.0,0.125


In [17]:
pos = []
neg = []

for num, row in lexiconfile.iterrows():
    # add positives
    if row['PosScore'] > 0.6 and row['NegScore'] < 0.2:
        words = row['lemma'].split()
        for word in words: 
            word = word.rstrip('0123456789#')
            if '_' not in word:
                pos.append(word)
    # add negatives
    elif row['NegScore'] > 0.6 and row['PosScore'] < 0.2:
        words = row['lemma'].split()
        for word in words: 
            word = word.rstrip('0123456789#')
            if '_' not in word:
                neg.append(word)

print(len(pos), len(neg))

1858 4367


In [18]:
# Check if words occur in both lists, if they do check how many times
# if word occur more in one list remove in other, if equal remove from both lists


new_pos = []
new_neg = []

for word in pos:
    if word in neg:
        if pos.count(word) > neg.count(word):
            if word not in new_pos:
                new_pos.append(word)
        elif neg.count(word) > pos.count(word):
            if word not in new_neg:
                new_neg.append(word)
    else:
        if word not in new_pos:
            new_pos.append(word)

for word in neg:
    if word not in new_neg and word not in pos:
        new_neg.append(word)

In [19]:
print(len(new_pos),len(new_neg))

1245 2765


In [20]:
# Add _pos to positive words

new_pos_comments = []
changed_words_pos = []

for line in commentlist_pos_test:
    new_line = []
    for word in line:
        if word in new_pos:
            
            new_word = word +"_pos"
            
#             count_changes_pos[word] += 1

            new_line.append(new_word)
            changed_words_pos.append(word)
            
            
            
        else:
            new_line.append(word)
    new_pos_comments.append(new_line)

print(new_pos_comments[:10])
print(len(changed_words_pos))

[['ieri', 'mattina', 'a', 'mio', 'figlio', 'di', '9anni', 'è', 'caduto', 'il', 'clempad', '4.4', '6-12', 'anni', 'regalato', 'da', 'babbo', 'n.il', 'dispay', 'esternamente', 'non', 'si', 'è', 'rotto', 'ma_pos', 'si', 'vede', 'una', 'riga', 'interna', 'e', 'il', 'video', 'ora', 'è', 'a', 'metà,neanche', 'il', 'tempo_pos', 'di', 'vedere', 'come_pos', 'funziona.come', 'posso', 'fare...(lui', 'è', 'disperato!!!!!)', 'vado', 'dal', 'rivenditore', 'oppure', 'mi', 'può', 'consigliare', 'come_pos', 'muovermi', 'per_pos', 'una', 'riparazione', 'più', 'rapida', 'possibile,grazie.andrea'], ['bellissima', 'recensione!!!!!', 'utile', 'e', 'completa.....è', 'stato', 'un', 'piacere_pos', 'guardarla', 'aspetto', "anch'io", 'quella', 'del', 'clempad', '4.4', 'xl', '6-12', 'anni....utile', 'anche', "l'informazione", 'sulla', 'scheda', 'sd.....alla', 'prossima!!!!!', ';-)'], ['aspetto', 'la', 'recensione', 'del', 'clempad', 'xl'], ['ahahhahaha', 'grande_pos', 'patrix!!', '=)'], ['bellissima', 'davvero!',

In [21]:
count_changes_pos = Counter(changed_words_pos)

In [22]:
# Add _neg to positive words

new_neg_comments = []
changed_words_neg = []
count_changes_neg = 0

for line in commentlist_neg_test:
    new_line = []
    for word in line:
        if word in new_neg:
            new_word = word +"_neg"
            count_changes_neg += 1
            new_line.append(new_word)
            changed_words_neg.append(word)
        else:
            new_line.append(word)
    
    new_neg_comments.append(new_line)

print(new_neg_comments[:10])
print(len(changed_words_neg))

[['io', 'ho', 'cancellato', "l'errore", 'avaria', 'motore', 'dovuto', 'alla', 'valvola', 'egr', '(dopo', 'averla', 'pulita)', 'con', 'questo', 'trasmettitore...risparmiato', 'un', 'sacchissimo', 'di', 'soldi...'], ['bellissimo', 'accessorio_neg', 'ma', 'fate', 'attenzione', 'soprattutto', 'pe', 'quelle', 'auto', 'ancora', 'in', 'garanzia', 'del', 'costruttore,', 'se', 'disgraziatamente_neg', 'rimanete', 'a', 'piedi', 'e', 'la', 'colpa_neg', 'è', 'di', 'questo', 'accessorio_neg', 'pagate', 'voi.', 'ocio'], ['ma', 'la', 'mia', 'meriva', 'del', '2012', 'ha', 'la', 'porta', 'obd?', 'e', 'simpatico', 'sto', 'coso'], ['quanta', 'energia', 'in', 'quel', 'puntocom', 'finale!', 'grande', 'gale'], ['e', 'cosi', 'tutti', 'quelli', 'che', 'faranno', 'la', 'scansione', 'dei', 'codici', "d'errore", 'si', 'spaventeranno', 'e', 'porteranno', 'la', 'macchina', 'dal', 'meccanico', 'pensando', 'che', 'abbia', 'qualcosa', 'che', 'non_neg', 'va', 'hahahahah'], ['video', 'molto', '"ondulato"', 'ahah', 'un',

In [23]:
count_changes_neg = Counter(changed_words_neg)

In [24]:
# Remove double words
words_neg = list(OrderedDict.fromkeys(changed_words_neg))
words_pos = list(OrderedDict.fromkeys(changed_words_pos))

In [25]:
print(len(words_pos))

385


In [26]:
pickle.dump(words_neg, open("changed_words_neg_ALL_IT.p", "wb" ))
pickle.dump(words_pos, open("changed_words_pos_ALL_IT.p", "wb" ))

In [27]:
new_data = new_neg_comments + new_pos_comments + commentlist_neu_test

In [28]:
# Count amount of times each changed word occurs in other context for weighting the embeddings

changed_word_neg_other_context = [] 
changed_word_pos_other_context = [] 

for line in new_data:
    for word in line:
        if word in words_neg:
            changed_word_neg_other_context.append(word)
        elif word in words_pos:
            changed_word_pos_other_context.append(word)

In [29]:
count_changes_neg_other = Counter(changed_word_neg_other_context)
count_changes_pos_other = Counter(changed_word_pos_other_context)

In [30]:
def getWeightdict(count_changes_pos,count_changes_pos_other):
    pos_word_weights = {}
    for word in count_changes_pos:
        pos_count = count_changes_pos[word]
        other_count = count_changes_pos_other[word]
        total_count_pos = pos_count + other_count
        weight_pos = pos_count / total_count_pos
        weight_other_pos = other_count / total_count_pos
        pos_word_weights[word] = [weight_pos,weight_other_pos]
    return pos_word_weights 

In [31]:
pos_word_weights = getWeightdict(count_changes_pos,count_changes_pos_other)

In [32]:
neg_word_weights = getWeightdict(count_changes_neg,count_changes_neg_other)

In [33]:
print(commentlist_neg_test[:10])

[['io', 'ho', 'cancellato', "l'errore", 'avaria', 'motore', 'dovuto', 'alla', 'valvola', 'egr', '(dopo', 'averla', 'pulita)', 'con', 'questo', 'trasmettitore...risparmiato', 'un', 'sacchissimo', 'di', 'soldi...'], ['bellissimo', 'accessorio', 'ma', 'fate', 'attenzione', 'soprattutto', 'pe', 'quelle', 'auto', 'ancora', 'in', 'garanzia', 'del', 'costruttore,', 'se', 'disgraziatamente', 'rimanete', 'a', 'piedi', 'e', 'la', 'colpa', 'è', 'di', 'questo', 'accessorio', 'pagate', 'voi.', 'ocio'], ['ma', 'la', 'mia', 'meriva', 'del', '2012', 'ha', 'la', 'porta', 'obd?', 'e', 'simpatico', 'sto', 'coso'], ['quanta', 'energia', 'in', 'quel', 'puntocom', 'finale!', 'grande', 'gale'], ['e', 'cosi', 'tutti', 'quelli', 'che', 'faranno', 'la', 'scansione', 'dei', 'codici', "d'errore", 'si', 'spaventeranno', 'e', 'porteranno', 'la', 'macchina', 'dal', 'meccanico', 'pensando', 'che', 'abbia', 'qualcosa', 'che', 'non', 'va', 'hahahahah'], ['video', 'molto', '"ondulato"', 'ahah', 'un', "po'", 'di', 'mal',

In [34]:
print(len(new_data))

217602


In [36]:
# Train both skip models

# model_SKIP = Word2Vec(new_data, size=300, sg=1)
model_SKIP_n10 = Word2Vec(new_data, size=300, sg=1, negative=10)

In [39]:
model_SKIP_n10.most_similar("fantastico_pos")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('stupendo_pos', 0.8124523162841797),
 ('spettacolare!', 0.8005304932594299),
 ('simpatico_pos', 0.7953038215637207),
 ('fantastico', 0.7924375534057617),
 ('divertentissimo', 0.7803273797035217),
 ('favoloso', 0.7680243253707886),
 ('carino_pos', 0.7674349546432495),
 ('bellissimo!', 0.7673802375793457),
 ('grandeeee', 0.7631446123123169),
 ('fantastico!', 0.7628900408744812)]

In [40]:
# 55976
# 115221
model_SKIP_n10.wv.vectors.shape[0]

32531

In [41]:
model_SKIP_n10.save("SKIP_negative10_IT_Change_inputSentiment_no_delete_test.model")
# model_SKIP.save("SKIP_IT_Change_inputSentiment_no_delete_test.model")

In [47]:
# Delete other context words

model_dict_delete = {}

count_word_del = 0
count_all = 0
count_else = 0
amount_posneg = 0
indict = 0


for word in model_SKIP_n10.wv.vocab:
    count_all +=1
    if word[-4:] == "_neg" or word[-4:] == "_pos":
        amount_posneg +=1
        non_word = word[:-4]
        if non_word not in model_dict_delete:
            model_dict_delete[non_word] = model_SKIP_n10.wv[word]
        else:
            print(word)
    else:
        if word not in words_neg and word not in words_pos:
            count_else +=1
            if word not in model_dict_delete:
                model_dict_delete[word] = model_SKIP_n10.wv[word]
            else:
                print(word)
                indict  +=1
        else:
            count_word_del +=1

print(count_all)
print("")   
print(count_word_del)
print(amount_posneg)
print(count_else)
print("")
print(count_word_del + amount_posneg + count_else)

32531

587
345
31599

32531

0
0


In [48]:
print(len(model_dict_delete))

31944


In [49]:
print(model_dict_delete["fantastico"])

[ 2.89118081e-01 -3.79209787e-01 -1.96773708e-01 -5.39195761e-02
 -3.05712014e-01  3.67750637e-02  3.77034873e-01  4.96099412e-01
  3.72668020e-02  2.08207574e-02  2.94383578e-02 -3.34917873e-01
 -1.35159222e-02 -1.54792219e-01  2.40416452e-01 -8.47082511e-02
 -1.76645085e-01  2.66523138e-02 -2.33833808e-02  1.07569769e-02
 -1.23223104e-01 -1.06809728e-01 -8.88472274e-02  2.82694101e-01
 -8.40720385e-02 -1.27765313e-01 -1.42776310e-01  1.52525127e-01
 -2.33185112e-01  3.40788543e-01  6.08045831e-02 -4.95503247e-02
  3.18533480e-01 -6.33776858e-02  3.37480873e-01  1.57377794e-01
 -8.49978402e-02  2.16086075e-01  1.34160668e-01  9.11097899e-02
  5.09929717e-01  7.27943331e-02 -1.87426582e-01  1.40792102e-01
  1.23210721e-01 -2.98863743e-02 -5.39682843e-02 -1.10212363e-01
  2.01096326e-01  2.63399214e-01  1.58809528e-01  3.90238464e-02
  3.06568444e-01  5.47487587e-02  1.17591456e-01  1.37621969e-01
 -7.25888759e-02 -1.97256535e-01 -1.33607060e-01  8.90514553e-02
  1.08468220e-01 -6.68198

In [50]:
pickle.dump(model_dict_delete, open("SKIP_negative10_IT_Change_inputSentiment_deletedoubles.p", "wb" ))

In [53]:
# Average the embeddings

model_dict_average = {}

count_word_del = 0
count_all = 0
count_else = 0
amount_posneg = 0
indict = 0


for word in model_SKIP_n10.wv.vocab:
    count_all +=1
    if word[-4:] == "_neg" or word[-4:] == "_pos":
        amount_posneg +=1
        non_word = word[:-4]
        
        vectorslist = []
        posneg_vector = model_SKIP_n10.wv[word]
        vectorslist.append(posneg_vector)
        
        try:
            other_vector = model_SKIP_n10.wv[non_word]
            vectorslist.append(other_vector)
            
            vectorarray = np.array(vectorslist)
            averagevector = np.average(vectorarray, axis=0)
            
            model_dict_average[non_word] = model_SKIP_n10.wv[averagevector]
            
        except:
            # use posneg word
            model_dict_average[non_word] = model_SKIP_n10.wv[word]
    
    else:
        if word not in words_neg and word not in words_pos:
            count_else +=1
            if word not in model_dict_average:
                model_dict_average[word] = model_SKIP_n10.wv[word]
            else:
                print(word)
                indict  +=1
        else:
            count_word_del +=1
        
        
            
        

print(count_all)
print("")   
print(count_word_del)
print(amount_posneg)
print(count_else)
print("")
print(count_word_del + amount_posneg + count_else)

32531

587
345
31599

32531

0
0


In [54]:
pickle.dump(model_dict_average, open("SKIP_negative10_IT_Change_inputSentiment_average.p", "wb" ))

In [57]:
# Weight and sum embeddings

model_dict_weightsum = {}

count_word_del = 0
count_all = 0
count_else = 0
amount_posneg = 0
indict = 0

for word in model_SKIP_n10.wv.vocab:
    count_all +=1
    if word[-4:] == "_neg" or word[-4:] == "_pos": 
        amount_posneg +=1
        non_word = word[:-4]
        
        if word[-4:] == "_neg":
            sentiment_weight, other_weight = neg_word_weights[non_word]
        else:
            sentiment_weight, other_weight = pos_word_weights[non_word]
        
        vectorslist = []
        posneg_vector = model_SKIP_n10.wv[word]
        vectorslist.append(posneg_vector*sentiment_weight)
        
        try:
            other_vector = model_SKIP_n10.wv[non_word]
            vectorslist.append(other_vector*other_weight)
            
            vectorarray = np.array(vectorslist)
            weight_sum_vector = np.sum(vectorarray, axis=0)
            
            model_dict_weightsum[non_word] = model_SKIP_n10.wv[weight_sum_vector]
        except:
            # use posneg word
            model_dict_weightsum[non_word] = model_SKIP_n10.wv[word]
    
    else:
        if word not in words_neg and word not in words_pos:
            count_else +=1
            if word not in model_dict_weightsum:
                model_dict_weightsum[word] = model_SKIP_n10.wv[word]
            else:
                print(word)
                indict  +=1
        else:
            count_word_del +=1
        
        
            
print(count_all)
print("")   
print(count_word_del)
print(amount_posneg)
print(count_else)
print("")
print(count_word_del + amount_posneg + count_else)


32531

587
345
31599

32531

0
0


In [58]:
pickle.dump(model_dict_weightsum, open("SKIP_negative10_IT_Change_inputSentiment_weightsum.p", "wb" ))